<a href="https://colab.research.google.com/github/ilinashah177/Coursework-Website/blob/main/Stepwise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install statsmodels pandas numpy

import pandas as pd
import numpy as np
import statsmodels.api as sm

In [4]:
from google.colab import files

# Upload the file
uploaded = files.upload()

Saving Stepwise.xlsx to Stepwise.xlsx


In [5]:
!pip install openpyxl
import pandas as pd

#Name of uploaded file
file_name = 'Stepwise.xlsx'

#Read data from the required sheet
df = pd.read_excel('Stepwise.xlsx', engine='openpyxl') # Use pd.read_excel and specify the engine

df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df

,Year,BRI Investments (USD millions),Total Energy Capacity Added from Completed CPEC Projects (MW),Total electricity consumption (GWh),Access to electricity (% of population),Annual change in coal energy consumption (TWh),Annual change in renewable energy consumption (TWh),Annual CO2 emissions (millon tonnes)
0,2015,520.40,400,85818,91.0,0.0,1.96,150.59
1,2016,893.25,650,90431,91.8,7.9,6.75,178.03
2,2017,2296.00,1469,95529,92.7,21.0,-7.66,218.36
3,2018,2062.00,1420,106928,93.4,54.5,0.43,210.35
4,2019,2907.00,1980,109461,93.9,18.9,16.68,207.62
5,2020,1600.00,701,108371,94.5,25.2,8.71,230.75
6,2021,2400.00,1100,116816,94.9,19.0,-4.51,247.90
7,2022,2716.00,1380,111263,95.0,-22.2,-4.25,223.83
8,2023,1440.00,1320,114300,95.6,-7.1,6.55,200.67


In [6]:
#Define dependent variable (environmental indicator, e.g., CO2 emissions)
y = df['Annual CO2 emissions (millon tonnes)']  # This is your dependent variable

#Define the independent variables (explanatory variables)
X = df[['BRI Investments (USD millions)', 'Total Energy Capacity Added from Completed CPEC Projects (MW)', 'Total electricity consumption (GWh)',
          'Access to electricity (% of population)', 'Annual change in coal energy consumption (TWh)', 'Annual change in renewable energy consumption (TWh)']]

#Add a constant term (intercept) to the model
X = sm.add_constant(X)

In [8]:
#Function for backward elimination
def backward_elimination(X, y, significance_level=0.05):
    # Start with all predictors
    X_selected = X.copy()

    while True:
        # Fit the model with the current predictors
        model = sm.OLS(y, X_selected).fit()

        # Get the p-values of the variables
        p_values = model.pvalues[1:]  # Exclude intercept

        # Check for the highest p-value
        max_p_value = p_values.max()

        # If the highest p-value is greater than the significance level, remove the variable
        if max_p_value > significance_level:
            excluded_var = p_values.idxmax()
            X_selected = X_selected.drop(columns=[excluded_var])
            print(f"Removed variable: {excluded_var} with p-value: {max_p_value:.4f}")
        else:
            # If all variables have p-values below the significance level, break the loop
            break

    return model

In [9]:
# Perform backward elimination
final_model = backward_elimination(X, y)

# Print the summary of the final model
print("\nFinal Model Summary:")
print(final_model.summary())

Removed variable: Annual change in renewable energy consumption (TWh) with p-value: 0.5909
Removed variable: Total electricity consumption (GWh) with p-value: 0.2721
Removed variable: Annual change in coal energy consumption (TWh) with p-value: 0.0982
Removed variable: Total Energy Capacity Added from Completed CPEC Projects (MW) with p-value: 0.0961
Removed variable: BRI Investments (USD millions) with p-value: 0.1235

Final Model Summary:
                                     OLS Regression Results                                     
Dep. Variable:     Annual CO2 emissions (millon tonnes)   R-squared:                       0.574
Model:                                              OLS   Adj. R-squared:                  0.513
Method:                                   Least Squares   F-statistic:                     9.418
Date:                                  Thu, 23 Jan 2025   Prob (F-statistic):             0.0181
Time:                                          15:38:26   Log-Likeliho

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)


In [10]:
#Function to perform Stepwise Regression
def stepwise_regression(X, y, direction="both", threshold_in=0.05, threshold_out=0.10):
    """
    Performs stepwise regression using both forward and backward elimination.
    """
    initial_list = X.columns.tolist()
    while True:
        changed = False
        #Forward Step: Adding variables to the model
        if direction in ["both", "forward"]:
            model = sm.OLS(y, X[initial_list]).fit()
            p_values = model.pvalues[1:]  # Exclude intercept
            max_p_value = p_values.max()
            if max_p_value > threshold_in:
                excluded_var = p_values.idxmax()
                initial_list.remove(excluded_var)
                changed = True

        #Backward Step: Removing variables from the model
        if direction in ["both", "backward"]:
            model = sm.OLS(y, X[initial_list]).fit()
            p_values = model.pvalues[1:]  # Exclude intercept
            max_p_value = p_values.max()
            if max_p_value > threshold_out:
                excluded_var = p_values.idxmax()
                initial_list.remove(excluded_var)
                changed = True

        # If no change, break the loop
        if not changed:
            break

    # Final model with selected variables
    final_model = sm.OLS(y, X[initial_list]).fit()
    return final_model

# Perform stepwise regression
stepwise_model = stepwise_regression(X, y)

# Print the summary of the final model
print(stepwise_model.summary())

                                     OLS Regression Results                                     
Dep. Variable:     Annual CO2 emissions (millon tonnes)   R-squared:                       0.574
Model:                                              OLS   Adj. R-squared:                  0.513
Method:                                   Least Squares   F-statistic:                     9.418
Date:                                  Thu, 23 Jan 2025   Prob (F-statistic):             0.0181
Time:                                          15:38:35   Log-Likelihood:                -38.704
No. Observations:                                     9   AIC:                             81.41
Df Residuals:                                         7   BIC:                             81.80
Df Model:                                             1                                         
Covariance Type:                              nonrobust                                         
                              

/usr/local/lib/python3.11/dist-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  res = hypotest_fun_out(*samples, **kwds)
